In [94]:
from bs4 import BeautifulSoup
import requests
import json
import math
import time
import pandas as pd
from nested_lookup import nested_lookup
from tqdm import tqdm
from requests_ip_rotator import ApiGateway



In [92]:
def parseNextCache(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html') 
    script = soup.find('script', {'id': '__NEXT_DATA__'})
    data = json.loads(script.text)
    return data

In [105]:
def parseNextCache(url, session):
    page = session.get(url)
    soup = BeautifulSoup(page.text, 'html') 
    script = soup.find('script', {'id': '__NEXT_DATA__'})
    data = json.loads(script.text)
    return data

In [116]:
def parseIndexPages(url):

    # # Create gateway object and initialise in AWS
    gateway = ApiGateway(url)
    gateway.start()

    # Assign gateway to session
    session = requests.Session()
    session.mount(url, gateway)

    data = parseNextCache(url, session)
    
    _first_page_results = nested_lookup("results", data)[0]
    _paging_info = _first_page_results["pageInfo"]
    total_pages = _paging_info["pageCount"] or math.ceil(_paging_info["total"] / _paging_info["limit"])
    product_previews = [edge["node"] for edge in _first_page_results["edges"]]
    product_sales = [salesInfo['lastSale'] for salesInfo in nested_lookup("salesInformation", data)]

    for i in tqdm(range(total_pages)):
        current_url = f"{url}&page={i+2}"
        data = parseNextCache(current_url, session)
        _page_results = nested_lookup("results", data)[0]
        product_previews.extend([edge["node"] for edge in _page_results["edges"]])
        product_sales.extend([salesInfo['lastSale'] for salesInfo in nested_lookup("salesInformation", data)])
    
    gateway.shutdown()
    return product_previews, product_sales
    


In [117]:

url = "https://stockx.com/search/sneakers/top-selling?s=indigo"
preview, sales = parseIndexPages(url)
df  = pd.DataFrame(preview)
df['last sale'] = sales
df

Starting API gateways in 10 regions.
Using 10 endpoints with name 'https://stockx.com/search/sneakers/top-selling?s=indigo - IP Rotate API' (10 new).


100%|██████████| 12/12 [00:06<00:00,  1.78it/s]


,id,name,urlKey,title,brand,description,model,condition,productCategory,listingType,gender,browseVerticals,media,favorite,productTraits,variants,market,last sale
0,ab307717-6fec-47f9-a09a-0713df18231b,Indigo Haze (Women's),nike-dunk-low-indigo-haze-womens,Nike Dunk Low Indigo Haze (Women's),Nike,The Nike Dunk Low Indigo Haze W is an addition...,Nike Dunk Low,New,sneakers,STANDARD,women,[sneakers],{'thumbUrl': 'https://images.stockx.com/images...,None,"[{'name': 'Retail Price', 'value': '110'}, {'n...",[{'id': '959a428f-8336-44d4-85a8-3005e7be9527'...,"{'currencyCode': 'USD', 'bidAskData': {'lowest...",130
1,5411080a-eb5e-4dda-9c83-6f4a0ab0d6ef,Indigo Haze (Women's),air-jordan-5-retro-low-indigo-haze-w,Jordan 5 Retro Low Indigo Haze (Women's),Jordan,,Jordan 5 Retro Low,New,sneakers,STANDARD,women,[sneakers],{'thumbUrl': 'https://images.stockx.com/images...,None,"[{'name': 'Retail Price', 'value': '180'}, {'n...",[{'id': 'fa4320b5-6235-40c7-9faf-976b3fe9023c'...,"{'currencyCode': 'USD', 'bidAskData': {'lowest...",258
2,2824b3f9-4848-4fe2-963f-c8e83f0a7e11,White Natural Indigo Burgundy,new-balance-550-white-maroon-natural-indigo,New Balance 550 White Natural Indigo Burgundy,New Balance,The New Balance 550 White Natural Indigo Burgu...,New Balance 550,New,sneakers,STANDARD,men,[sneakers],{'thumbUrl': 'https://images.stockx.com/images...,None,"[{'name': 'Retail Price', 'value': '110'}, {'n...",[{'id': '9ff0c39d-5091-4f8c-afcd-72a822bd2cb4'...,"{'currencyCode': 'USD', 'bidAskData': {'lowest...",88
3,cfb37c62-af3c-4b8c-bbd2-14b656b00ba0,Vintage Indigo Beige,new-balance-550-vintage-indigo-beige,New Balance 550 Vintage Indigo Beige,New Balance,The New Balance 550 Vintage Indigo Beige is a ...,New Balance 550,New,sneakers,STANDARD,men,[sneakers],{'thumbUrl': 'https://images.stockx.com/images...,None,"[{'name': 'Retail Price', 'value': '110'}, {'n...",[{'id': '1e9ae124-4b34-46db-a6ea-360d093459d4'...,"{'currencyCode': 'USD', 'bidAskData': {'lowest...",107
4,c8535733-1136-49c1-9991-2fd2e41ac925,Indigo,air-jordan-12-retro-stone-blue,Jordan 12 Retro Indigo,Jordan,Ever since Michael Jordan’s memorable 1997 Flu...,Jordan 12 Retro,New,sneakers,STANDARD,men,[sneakers],{'thumbUrl': 'https://images.stockx.com/images...,None,"[{'name': 'Retail Price', 'value': '190'}, {'n...",[{'id': '94689371-dce0-4789-9b00-cef3dd2cd6e9'...,"{'currencyCode': 'USD', 'bidAskData': {'lowest...",271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,086cf726-7b88-47a1-ba13-1dc0383e8f63,Legacy Indigo,adidas-adizero-prime-legacy-indigo,adidas Adizero Prime X Legacy Indigo,adidas,,adidas Adizero Prime X,New,sneakers,STANDARD,men,[sneakers],{'thumbUrl': 'https://images.stockx.com/images...,None,"[{'name': 'Retail Price', 'value': '250'}]",[{'id': '8e30c28d-d4c7-4c53-a8e0-b713628fc0cb'...,"{'currencyCode': 'USD', 'bidAskData': {'lowest...",177
516,9c13689c-16b2-454d-a523-7612fa0aad7f,Sky Blue Natural Indigo,new-balance-327-sky-blue-natural-indigo,New Balance 327 Sky Blue Natural Indigo,New Balance,,New Balance 327,New,sneakers,STANDARD,unisex,[sneakers],{'thumbUrl': 'https://images.stockx.com/images...,None,"[{'name': 'Retail Price', 'value': '100'}]",[{'id': '82be51a3-a0cf-49a1-b0b0-2a6b3afddcbd'...,"{'currencyCode': 'USD', 'bidAskData': {'lowest...",45
517,51a0b349-f905-4c69-a1d0-f83c61d0cc12,Cloud White Tech Indigo,adidas-top-ten-2000-cloud-white-tech-indigo,adidas Top Ten 2000 Cloud White Tech Indigo,adidas,,adidas Top Ten 2000,New,sneakers,STANDARD,men,[sneakers],{'thumbUrl': 'https://images.stockx.com/images...,None,"[{'name': 'Retail Price', 'value': '130'}, {'n...",[{'id': '132e8b67-0549-4f37-8bce-209f7d04a2da'...,"{'currencyCode': 'USD', 'bidAskData': {'lowest...",150
518,ae3b0c6b-151e-4a79-b02f-b85aba634cfe,Legacy Indigo Turbo,adidas-ultra-boost-22-legacy-indigo-turbo,adidas Ultra Boost 22 Legacy Indigo Turbo,adidas,,adidas Ultra Boost 22,New,sneakers,STANDARD,men,[sneakers],{'thumbUrl': 'https://images.stockx.com/images...,None,"[{'

Check if gateways are open: aws apigateway get-rest-apis --output text

In [118]:
# run to close gateways
gateway = ApiGateway("https://stockx.com/search/sneakers/top-selling?s=indigo")
gateway.shutdown()

Deleting gateways for site 'https://stockx.com/search/sneakers/top-selling?s=indigo'.
Deleted 10 endpoints with for site 'https://stockx.com/search/sneakers/top-selling?s=indigo'.


['5ze6q7wxhd',
 'rtvp6qt67i',
 'tx28wwmntb',
 'c6j93974k7',
 '4vh4di1yk5',
 'd90lz0it88',
 'h0mk1zj2ne',
 'b55lvyp321',
 'bfcjd2400k',
 '7axfh6hkkh']